In [ ]:
# run(`conda create --channel conda-forge --channel bioconda --channel defaults --strict-channel-priority --name blast blast`)

In [ ]:
import Pkg
pkgs = [
    "Revise",
    "DataFrames",
    "uCSV",
    "StatsPlots"
]
# Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end
import Mycelia

In [ ]:
data_dir = joinpath(dirname(pwd()), "data")

In [ ]:
total_mem = 64
threads = max(1, Int(total_mem//8))
mem_per_cpu = Int(total_mem//threads)

In [ ]:
db_fasta = joinpath(data_dir, "results/high-confidence-hiv-contigs.fna.gz")
db = "hiv-high-confidence"

# db_fasta = joinpath(data_dir, "results/high-confidence-viral-contigs.fna.gz")
# db = "viral-high-confidence"

blast_task = "megablast"
blastdbs_dir = mkpath(joinpath(homedir(), "workspace", "blastdbs"))
blast_db = joinpath(blastdbs_dir, db)
# build new db with all of the input sequences from all of the assemblies
run(pipeline(`gzip -dc $(db_fasta)`, `conda run --live-stream -n blast makeblastdb -in - -out $(blast_db) -parse_seqids -title "$(db)" -dbtype nucl`))
# conda run -n blast update_blastdb.pl --showall
# Mycelia.download_blast_db(db=db, outdir=blastdbs_dir, source="ncbi", conda_env="blast")

In [ ]:
fasta_paths = filter(x -> !occursin(".ipynb_checkpoints", x) && occursin(r"\.fasta$", x), readdir(joinpath(data_dir, "exposome_data", "joint-reads"), join=true))
# sample_paths = filter(x -> isfile(joinpath.(x, "megahit", "final.contigs.fastg.gfa.fna")), sample_paths)

In [ ]:
for fasta_path in fasta_paths
    # sample = basename(fasta_path)
    blastn_directory = dirname(fasta_path)
    assembled_fasta = fasta_path
    target_outfile = joinpath(blastn_directory, basename(assembled_fasta) * ".blastn.$(db).$(blast_task).txt")
    # # --mem=$(memory)G
    # if isfile(target_outfile)
    #         # @show sample, filesize(target_outfile)
    #     push!(file_sizes, filesize(target_outfile))
    # else
    #     @show sample
    # end

    cmd = "julia $(homedir())/workspace/Mycelia/scripts/run-blastn.jl --fasta $(assembled_fasta) --blast_db $(blast_db) --task $(blast_task) --out_directory $(blastn_directory) --threads $(threads)"
    submission = 
    `sbatch 
    --mail-user=cameron.prybol@gmail.com
    --mail-type=ALL
    --error=%x-%j.err
    --output=%x-%j.out
    --partition=batch
    --account=mpsnyder
    --nodes=1
    --ntasks=1
    --time=1-00:00:00    
    --cpus-per-task=$(threads)
    --mem-per-cpu=$(mem_per_cpu)G
    --job-name=blastn
    --wrap $cmd
    `
    run(submission)
    # don't submit too quickly! We can hit issues when the jobs all start at once due to so many processes hitting the same databases
    # 10 felt too slow, some jobs finish in ~10-15 seconds
    sleep(1)
end